In [433]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('elasticnet')
library(elasticnet)
#install.packages('kernlab')
library(kernlab)
#install.packages('e1071')
library(e1071)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)
library(mediation)
library(lavaan)
library(lavaanPlot)
library(plotly)
library(MVN)
#install.packages('MissMech')
library(semTools)
library(MissMech)
#install.packages('aod')
library(aod)


# Read in the data 

In [576]:
#di=read.table('/dagher/dagher11/filip/PRSOB/data/brain_data.csv', header=TRUE, na.strings = "", sep=',', quote = "\"")
cov19=read.table('/dagher/dagher11/filip/PRSOB/data/covid19_result.txt', header=TRUE, na.strings = "", sep='\t', quote = "\"")

covidinf=cov19
cov19$date=as.Date( as.character(cov19$specdate), "%d/%m/%y")
cov19 <- subset(cov19, date <= as.Date("2020-04-26") ) # filter participants before April 26th
cov19=cov19[!duplicated(cov19$eid),]

## Add hypertension and diabetes diagnosis

In [486]:
di$hypertension <- 0
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', colnames(di)))) { 
    di$hypertension[di[i] == 'High blood pressure'] <- 1 # Hypertension
}

# Mark participants who have diabetes - self-reported
di$diabetes <- 0
for (i in as.numeric(grep('diabetes_diagnosed_by_doctor', colnames(di)))) { 
    di$diabetes[di[i] =="Yes"] <- 1
}

# Mark participants with hypertension - diagnosed
di$hypertension2 <- 0
for (i in as.numeric(grep('icd10', colnames(di)))) { 
    di$hypertension2[di[i] == 'I10 Essential (primary) hypertension'] <- 1 # Hypertension
    }

# Mark participants who have diabetes - diagnoses
exclusions_diabetes=c('^E10','^E11','^E12','^E13','^E14')
di$diabetes2 <- NA
for (i in as.numeric(grep('icd10', colnames(di)))) {
    for (d in 1:length(exclusions_diabetes)) {
        di$diabetes2[grepl(exclusions_diabetes[d], di[[i]])] = 1
    }
}

di$Hypertension=0
di$Hypertension[di$hypertension=='1' | di$hypertension2=='1']=1
di$Diabetes=0
di$Diabetes[di$diabetes=='1' | di$diabetes2=='1']=1

# MetS latent variable and OR - covid infection

In [577]:
covidinf=merge(di,covidinf,by='eid')

In [968]:
covidinf$TG=scale(log(covidinf$triglycerides_30870.0.0))
covidinf$HDL=scale(log(covidinf$hdl_cholesterol_30760.0.0))
covidinf$Sex=as.factor(covidinf$sex_31.0.0)
covidinf$Waist=scale(covidinf$waist_circumference_48.0.0)
covidinf$HbA1c=scale(log(covidinf$glycated_haemoglobin_hba1c_30750.0.0))
covidinf$systolicBP=scale(rowMeans(cbind(covidinf$systolic_blood_pressure_automated_reading_4080.0.0,
                                      covidinf$systolic_blood_pressure_automated_reading_4080.0.1)))
covidinf$diastolicBP=scale(rowMeans(cbind(covidinf$diastolic_blood_pressure_automated_reading_4079.0.0,
                                       covidinf$diastolic_blood_pressure_automated_reading_4079.0.1)))
covidinf$glucose=scale(log(covidinf$glucose_30740.0.0))


COVinf=select(covidinf, TG, HDL, Sex, Hypertension, Diabetes, result,
          laboratory, origin, spectype, qualifications_6138.0.0, townsend_deprivation_index_at_recruitment_189.0.0,
          Waist, year_of_birth_34.0.0, smoking_status_20116.0.0,
          ethnic_background_21000.0.0, HbA1c, systolicBP, diastolicBP, glucose,
             average_total_household_income_before_tax_738.0.0, sex_31.0.0)



for (j in c(1, 2, 11, 12, 16, 17, 18, 19)) { # 
    Q3=as.numeric(quantile(COVinf[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(COVinf[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    COVinf[j][COVinf[j]<lower]=NA
    COVinf[j][COVinf[j]>upper]=NA
    
}

COVinf=na.omit(COVinf)

COVinf$Hypertension=as.numeric(COVinf$Hypertension)
COVinf$Diabetes=as.numeric(COVinf$Diabetes)

nrow(COVinf)

[1] 5163

In [969]:
COVID_infection = '
#Latent variables

BP =~ diastolicBP + systolicBP
HypertensionLatent =~  BP + Hypertension
DiaSer =~ HbA1c + glucose
DiabetesLatent =~ DiaSer + Diabetes

MetS =~ Waist + TG + HDL + HypertensionLatent + DiabetesLatent 


'

COVID_fitinf <- cfa(COVID_infection, data=COVinf, estimator='MLM')
summary(COVID_fitinf, standardized=TRUE, fit.measures=T)
#fitmeasures(WMHOB_fit) #fit indices

lavaan 0.6-7 ended normally after 68 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of free parameters                         22
                                                      
  Number of observations                          5163
                                                      
Model Test User Model:
                                              Standard      Robust
  Test Statistic                               964.760     889.752
  Degrees of freedom                                23          23
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.084
       Satorra-Bentler correction                                 

Model Test Baseline Model:

  Test statistic                              9928.177    8650.400
  Degrees of freedom                                36          36
  P-value                        

### Get latent variable scores

In [970]:
COVinf$MetS=(lavPredict(COVID_fitinf, newdata = NULL, type = "lv", method = "regression",
           se = "none", acov = "none", label = TRUE, fsm = FALSE, 
           append.data = FALSE, assemble = FALSE,
           level = 1L, optim.method = "bfgs", ETA = NULL))[,5]

### Run logistic regression

In [971]:
metsinf_logit=(glm(result ~ townsend_deprivation_index_at_recruitment_189.0.0 + 
           MetS +
           sex_31.0.0 +
           year_of_birth_34.0.0 +
           laboratory +
           spectype +
           qualifications_6138.0.0 + 
           smoking_status_20116.0.0 + 
           ethnic_background_21000.0.0 +
           average_total_household_income_before_tax_738.0.0, data=COVinf, family='binomial'(link="logit")))

summary(metsinf_logit)


Call:
glm(formula = result ~ townsend_deprivation_index_at_recruitment_189.0.0 + 
    MetS + sex_31.0.0 + year_of_birth_34.0.0 + laboratory + spectype + 
    qualifications_6138.0.0 + smoking_status_20116.0.0 + ethnic_background_21000.0.0 + 
    average_total_household_income_before_tax_738.0.0, family = binomial(link = "logit"), 
    data = COVinf)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9476  -0.8220  -0.6744   1.2349   2.2259  

Coefficients:
                                                                                 Estimate
(Intercept)                                                                    -26.885068
townsend_deprivation_index_at_recruitment_189.0.0                                0.016008
MetS                                                                             0.260427
sex_31.0.0Male                                                                   0.083398
year_of_birth_34.0.0                                                

### Get odds ratio with CIs

In [972]:
exp(cbind(coef(metsinf_logit), confint(metsinf_logit)))  

Waiting for profiling to be done...
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities nume

,,2.5 %,97.5 %
(Intercept),2.108451e-12,1.387308e-19,3.236991e-05
townsend_deprivation_index_at_recruitment_189.0.0,1.016137e+00,9.956684e-01,1.036940e+00
MetS,1.297484e+00,1.151999e+00,1.461532e+00
sex_31.0.0Male,1.086974e+00,9.434037e-01,1.252576e+00
year_of_birth_34.0.0,1.013375e+00,1.004848e+00,1.021968e+00
laboratory,1.013840e+00,1.011307e+00,1.016388e+00
spectype,1.012069e+00,9.982591e-01,1.025962e+00
qualifications_6138.0.0College or University degree,7.643182e-01,6.025842e-01,9.721560e-01
qualifications_6138.0.0CSEs or equivalent,7.843689e-01,5.585117e-01,1.097038e+00
qualifications_6138.0.0None of the above,1.082865e+00,8.410078e-01,1.398018e+00


# MetS latent variable and OR - covid hospitalisation

In [989]:
covid=merge(di,cov19,by='eid')

covid$TG=scale(log(covid$triglycerides_30870.0.0))
covid$HDL=scale(log(covid$hdl_cholesterol_30760.0.0))
covid$Sex=as.factor(covid$sex_31.0.0)
covid$Waist=scale(covid$waist_circumference_48.0.0)
covid$HbA1c=scale(log(covid$glycated_haemoglobin_hba1c_30750.0.0))
covid$systolicBP=scale(rowMeans(cbind(covid$systolic_blood_pressure_automated_reading_4080.0.0,
                                      covid$systolic_blood_pressure_automated_reading_4080.0.1)))
covid$diastolicBP=scale(rowMeans(cbind(covid$diastolic_blood_pressure_automated_reading_4079.0.0,
                                       covid$diastolic_blood_pressure_automated_reading_4079.0.1)))
covid$glucose=scale(log(covid$glucose_30740.0.0))

In [990]:
COV=select(covid, TG, HDL, Sex, Hypertension, Diabetes, result,
          laboratory, origin, spectype, qualifications_6138.0.0, townsend_deprivation_index_at_recruitment_189.0.0,
          Waist, year_of_birth_34.0.0, smoking_status_20116.0.0,
          ethnic_background_21000.0.0, HbA1c, systolicBP, diastolicBP, glucose,
             average_total_household_income_before_tax_738.0.0, sex_31.0.0)



for (j in c(1, 2, 11, 12, 16, 17, 18, 19)) { # 
    Q3=as.numeric(quantile(COV[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(COV[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    COV[j][COV[j]<lower]=NA
    COV[j][COV[j]>upper]=NA
    
}

COV=na.omit(COV)

COV$Hypertension=as.numeric(COV$Hypertension)
COV$Diabetes=as.numeric(COV$Diabetes)


In [991]:
nrow(COV)

[1] 1675

In [992]:
COVID_SEM = '
#Latent variables

BP =~ diastolicBP + systolicBP
HypertensionLatent =~  BP + Hypertension
DiaSer =~ HbA1c + glucose
DiabetesLatent =~ DiaSer + Diabetes

MetS =~ Waist + TG + HDL + HypertensionLatent + DiabetesLatent 


'

COVID_fit <- cfa(COVID_SEM, data=COV, estimator='MLM')
summary(COVID_fit, standardized=TRUE, fit.measures=T)
#fitmeasures(WMHOB_fit) #fit indices

lavaan 0.6-7 ended normally after 64 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of free parameters                         22
                                                      
  Number of observations                          1675
                                                      
Model Test User Model:
                                              Standard      Robust
  Test Statistic                               332.127     301.369
  Degrees of freedom                                23          23
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.102
       Satorra-Bentler correction                                 

Model Test Baseline Model:

  Test statistic                              3196.511    2773.359
  Degrees of freedom                                36          36
  P-value                        

### Get latent variable scores

In [993]:
COV$MetS=(lavPredict(COVID_fit, newdata = NULL, type = "lv", method = "regression",
           se = "none", acov = "none", label = TRUE, fsm = FALSE, 
           append.data = FALSE, assemble = FALSE,
           level = 1L, optim.method = "bfgs", ETA = NULL))[,5]

### Run logistic regression

In [997]:
mets_logit=(glm(result ~ townsend_deprivation_index_at_recruitment_189.0.0 + 
           MetS +
           sex_31.0.0 +
           year_of_birth_34.0.0 +
           laboratory +
           spectype +
           qualifications_6138.0.0 + 
           smoking_status_20116.0.0 + 
           ethnic_background_21000.0.0 +
           average_total_household_income_before_tax_738.0.0, data=COV, family='binomial'(link="logit")))

summary(mets_logit)


Call:
glm(formula = result ~ townsend_deprivation_index_at_recruitment_189.0.0 + 
    MetS + sex_31.0.0 + year_of_birth_34.0.0 + laboratory + spectype + 
    qualifications_6138.0.0 + smoking_status_20116.0.0 + ethnic_background_21000.0.0 + 
    average_total_household_income_before_tax_738.0.0, family = binomial(link = "logit"), 
    data = COV)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7128  -0.9554  -0.7544   1.2297   1.9781  

Coefficients:
                                                                                 Estimate
(Intercept)                                                                    -22.198621
townsend_deprivation_index_at_recruitment_189.0.0                               -0.002244
MetS                                                                             0.260454
sex_31.0.0Male                                                                   0.144583
year_of_birth_34.0.0                                                   

### Get odds ratio

In [998]:
exp(cbind(coef(mets_logit), confint(mets_logit)))  

Waiting for profiling to be done...
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities nume

,,2.5 %,97.5 %
(Intercept),2.286975e-10,2.948844e-22,1.737050e+02
townsend_deprivation_index_at_recruitment_189.0.0,9.977583e-01,9.638591e-01,1.032638e+00
MetS,1.297519e+00,1.053531e+00,1.599029e+00
sex_31.0.0Male,1.155558e+00,9.154548e-01,1.459179e+00
year_of_birth_34.0.0,1.011069e+00,9.970589e-01,1.025286e+00
laboratory,1.014747e+00,1.010305e+00,1.019256e+00
spectype,1.018918e+00,9.957144e-01,1.042557e+00
qualifications_6138.0.0College or University degree,9.032076e-01,6.133235e-01,1.337310e+00
qualifications_6138.0.0CSEs or equivalent,1.002676e+00,5.770640e-01,1.735276e+00
qualifications_6138.0.0None of the above,1.081426e+00,7.124286e-01,1.649607e+00
